In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm


import warnings
warnings.simplefilter("ignore")

In [14]:
BASE_PATH = "/Users/thibaut/Desktop"

In [15]:
df = pd.read_csv(f"{BASE_PATH}/230K_hw2/hw2_data_cleaned.csv", index_col=[0, 1])
df.index = pd.MultiIndex.from_arrays(
    [pd.to_datetime(df.index.get_level_values(0)).date, df.index.get_level_values(1)]
)

In [16]:
df

log_size    log_bm   log_pcf       mom     strev  \
           permno                                                      
1970-02-27 10006    5.603962 -0.229413  2.962434 -0.162378  0.051272   
           10102    5.226821 -0.235722  1.409767 -0.505290  0.004156   
           10137    6.108358 -0.462035  1.653455 -0.151756  0.123377   
           10145    6.480045 -0.295714  1.294727 -0.267678  0.012832   
           10153    5.542048 -0.018164  3.119630 -0.202092 -0.005209   
...                      ...       ...       ...       ...       ...   
2022-02-28 93369    8.931354 -2.900422  3.358220 -0.150411  0.143396   
           93374    8.899731 -0.223144  1.850500  0.379581 -0.100254   
           93423    8.232892 -3.411248  2.438688 -0.121693  0.105597   
           93429    9.433748 -1.220780  2.179739  0.191814 -0.006418   
           93436   13.709701 -3.352407  4.751545  0.326930 -0.070769   

                        vol    roa    roe  log_age_lb    price     log_to  \
           permno                                                           
1970-02-27 10006   0.065000  0.157  0.099    3.788477   48.125  15.291317   
           10102   0.051430  0.125  0.066    3.788477   16.875  14.632174   
           10137   0.052633  0.122  0.138    3.788477   21.625  14.852979   
           10145   0.048640  0.127  0.053    3.788477   23.500  15.986870   
           10153   0.084014 -0.100 -0.125    3.788477   23.875  15.421006   
...                     ...    ...    ...         ...      ...        ...   
2022-02-28 93369   0.104120  0.189  0.740    2.464378   27.190  20.269955   
           93374   0.066705  0.137  0.240    2.457362   67.040  20.731988   
           93423   0.113710  0.164  0.418    2.457362   43.660  21.073571   
           93429   0.060981  0.139  0.120    2.457362  117.290  21.136116   
           93436   0.134342  0.139  0.160    2.457362  870.430  26.717796   

                         rf        rm  ols_3m_d  ols_1y_d  ols_5y_m  \
           permno                                                     
1970-02-27 10006   0.006098  0.057959  0.629034  0.890264  1.079448   
           10102   0.006098  0.057959  1.562337  1.147668  1.005860   
           10137   0.006098  0.057959  0.941398  0.756613  0.717579   
           10145   0.006098  0.057959  1.214475  1.319379  0.997102   
           10153   0.006098  0.057959  0.713192  1.011173  1.017472   
...                     ...       ...       ...       ...       ...   
2022-02-28 93369   0.000000 -0.022821  0.855424  1.095643  1.004980   
           93374   0.000000 -0.022821  0.912535  0.911184  1.127878   
           93423   0.000000 -0.022821  1.174469  1.285446  1.172750   
           93429   0.000000 -0.022821  0.678829  0.549325  0.811812   
           93436   0.000000 -0.022821  2.227783  1.997363  1.086022   

                   f_ols_1y_d  
           permno              
1970-02-27 10006     0.707153  
           10102     1.339038  
           10137     0.643003  
           10145     1.239788  
           10153     1.737870  
...                       ...  
2022-02-28 93369     1.347405  
           93374     0.612557  
           93423     1.423048  
           93429     0.540210  
           93436     1.768254  

[813781 rows x 17 columns]

### We can create several predictive features

In [17]:
df["market_cap"] = np.exp(df["log_size"])
df["capm_return"] = df["ols_1y_d"]*(df["rm"]-df["rf"])

### The features below capture mean reversion

In [18]:
rel_diff_columns = ['log_size', 'log_bm', 'log_pcf', 'mom', 'strev', 'vol', 'roa', 'roe', 'log_to', 'ols_3m_d']
for col in rel_diff_columns:
    df[f'{col}_rel_diff'] = df[col] - df.groupby(level=0)[col].transform('mean')

### It makes sense to look not only at the level but also at the first difference of log_size, log_bm, log_pcf, log_age_lb, price, and log_to, as we want to track changes.

In [19]:
df["log_price"] = np.log(df["price"])
L_log_features = ["log_size", "log_bm", "log_pcf", "log_age_lb", "log_price", "log_to"]
for col in L_log_features: 
    df[f"{col}_diff"] = df[col].unstack().diff().stack()
df.dropna(inplace=True)

In [20]:
df.to_csv(f"{BASE_PATH}/230K_hw2/features.csv")